<a href="https://colab.research.google.com/github/santhimaddipudi/Azure-Databricks-project-on-Yelp-Dataset/blob/main/python_solve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# positive missing number


def solution(A):  # Our original array

    m = max(A)  # Storing maximum value
    if m < 1:
        # In case all values in our array are negative
        return 1
    if len(A) == 1:

        # If it contains only one element
        return 2 if A[0] == 1 else 1
    l = [0] * m
    for i in range(len(A)):
        if A[i] > 0:
            if l[A[i] - 1] != 1:

                # Changing the value status at the index of our list
                l[A[i] - 1] = 1
    for i in range(len(l)):

        # Encountering first 0, i.e, the element with least value
        if l[i] == 0:
            return i + 1
            # In case all values are filled between 1 and m
    return i + 2



In [ ]:

    arr = [13,14,16,20]
    print(solution(arr))

1


In [ ]:
def leastpositive(a):
  for i in range(1,len(a)+1):#1 to length of array
    if i not in a:
      return(i)
      break
    else:
      pass
  a1 = [1,2,3,4,5,6,7,8,9,10]
  print(leastpositive(a1))

In [ ]:
  a1 = [1,2,3,4,5,6,7,8,9,10]
  print(leastpositive(a1))

None


##aggegate and show as column headers


In [5]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=bb23d1bde558eb96d89b2d2d2b6c6feda5885743dfdbd1cd7962781628c0c1d6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession, DataFrame

In [ ]:
data=[
('Rudra','math',79),
('Rudra','eng',60),
('Shivu','math', 68),
('Shivu','eng', 59),
('Anu','math', 65),
('Anu','eng',80)
]
schema="Name string,Sub string,Marks int"
df=spark.createDataFrame(data=data,schema=schema)
df.show()

df_output=df.groupBy(col("Name")).agg(collect_list(col("Marks")).alias("Marks_New"))
df_output=df_output.select(col("Name"),col("Marks_New")[0].alias("math"),col("Marks_New")[1].alias("eng"))
df_output.show()

+-----+----+-----+
| Name| Sub|Marks|
+-----+----+-----+
|Rudra|math|   79|
|Rudra| eng|   60|
|Shivu|math|   68|
|Shivu| eng|   59|
|  Anu|math|   65|
|  Anu| eng|   80|
+-----+----+-----+

+-----+----+---+
| Name|math|eng|
+-----+----+---+
|Shivu|  68| 59|
|Rudra|  79| 60|
|  Anu|  65| 80|
+-----+----+---+



In [4]:
##DISPLAY Total dataframes in notebook
from pyspark.sql import dataframe
for k,v in globals().items():
  if(type(v)=dataframe):
    print(k)
    display(v) ##will display the dataframe contents

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (<ipython-input-4-8766544d4da7>, line 4)

##get origin and destination city only

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import min,max,col,row_number

In [ ]:
flights_data = [(1,'Flight1' , 'Delhi' , 'Hyderabad'),
                (1,'Flight2' , 'Hyderabad' , 'Kochi'),
                (1,'Flight3' , 'Kochi' , 'Mangalore'),
                (2,'Flight1' , 'Mumbai' , 'Ayodhya'),
                (2,'Flight2' , 'Ayodhya' , 'Gorakhpur')
                ]

_schema = "cust_id int, flight_id string , origin string , destination string"

df_flight = spark.createDataFrame(data = flights_data , schema= _schema)
df_flight.show()

+-------+---------+---------+-----------+
|cust_id|flight_id|   origin|destination|
+-------+---------+---------+-----------+
|      1|  Flight1|    Delhi|  Hyderabad|
|      1|  Flight2|Hyderabad|      Kochi|
|      1|  Flight3|    Kochi|  Mangalore|
|      2|  Flight1|   Mumbai|    Ayodhya|
|      2|  Flight2|  Ayodhya|  Gorakhpur|
+-------+---------+---------+-----------+



In [ ]:
#create row number
df_1=df_flight.withColumn("rn",row_number().over(Window.partitionBy(col("cust_id")).orderBy(col("flight_id"))))
df_1.show()

+-------+---------+---------+-----------+---+
|cust_id|flight_id|   origin|destination| rn|
+-------+---------+---------+-----------+---+
|      1|  Flight1|    Delhi|  Hyderabad|  1|
|      1|  Flight2|Hyderabad|      Kochi|  2|
|      1|  Flight3|    Kochi|  Mangalore|  3|
|      2|  Flight1|   Mumbai|    Ayodhya|  1|
|      2|  Flight2|  Ayodhya|  Gorakhpur|  2|
+-------+---------+---------+-----------+---+



In [ ]:
#find start and end flight
df_2=df_1.groupBy(col("cust_id")).agg(
    min(col("rn")).alias("start"),
    max(col("rn")).alias("end")
)
df_2.show()

+-------+-----+---+
|cust_id|start|end|
+-------+-----+---+
|      1|    1|  3|
|      2|    1|  2|
+-------+-----+---+



In [ ]:
#join twon dataframes to get start and end dates for each flight
df_answer=df_1.join(df_2,on=df_1.cust_id==df_2.cust_id).drop(df_1.cust_id)
df_answer.show()

+---------+---------+-----------+---+-------+-----+---+
|flight_id|   origin|destination| rn|cust_id|start|end|
+---------+---------+-----------+---+-------+-----+---+
|  Flight3|    Kochi|  Mangalore|  3|      1|    1|  3|
|  Flight2|Hyderabad|      Kochi|  2|      1|    1|  3|
|  Flight1|    Delhi|  Hyderabad|  1|      1|    1|  3|
|  Flight2|  Ayodhya|  Gorakhpur|  2|      2|    1|  2|
|  Flight1|   Mumbai|    Ayodhya|  1|      2|    1|  2|
+---------+---------+-----------+---+-------+-----+---+



In [ ]:
#group the data and get final output
df_final=df_answer.groupBy(col("cust_id")).agg(
    max(when(col("rn")==col("start"),col("origin"))).alias("origin"),
    max(when(col("rn")==col("end"),col("destination"))).alias("destination"),
)
df_final.show()

+-------+------+-----------+
|cust_id|origin|destination|
+-------+------+-----------+
|      1| Delhi|  Mangalore|
|      2|Mumbai|  Gorakhpur|
+-------+------+-----------+



##Largest subarray sum problem

In [ ]:
def maxSubArraySum(a,size):

    max_so_far =a[0]
    curr_max = a[0]

    for i in range(1,size):
        curr_max = max(a[i], curr_max + a[i])
        max_so_far = max(max_so_far,curr_max)

    return max_so_far

In [ ]:
# Driver function to check the above function
a = [-2,-3,4,3,-3,1,6,-3]
print ("Maximum contiguous sum is", maxSubArraySum(a,len(a)))

Maximum contiguous sum is 11


# Python program to find the smallest number evenly   divisible by all number 1 to n

In [ ]:

import math

# Returns the lcm of first n numbers
def lcm(n):
    ans = 1
    for i in range(1, n + 1):
        ans = int((ans * i)/math.gcd(ans, i))
    return ans

# main
n = 4
print (lcm(n))

12


# Python3 program to Find missing integers in list

In [ ]:
def find_missing(lst):
    max = lst[0]
    for i in lst:
        if i > max:
            max = i

    min = lst[0]
    for i in lst:
        if i < min:
            min = i

    list1 = []

    for num in range(min + 1, max):
        if num not in lst:
            list1.append(num)

    return list1
##another way
def find_missing_2(lst):
    sorted_list=sorted(lst)
    return (set(range(sorted_list[0], sorted_list[-1])) - set(lst))#set function will give the range of numbers from min and max range given

# Driver code
lst = [ 16,10,11,12,1,3,4,6,8]
print(find_missing(lst))
print(find_missing_2(lst))

[2, 5, 7, 9, 13, 14, 15]
{2, 5, 7, 9, 13, 14, 15}


##convert ecah word first char to capital in a column


In [32]:
from pyspark.sql.functions import lit, udf, col

In [73]:
data=[('1','santhi maddipudi'),('2','srujan veera'),(3,'dev veeramachaneni')]
column=['num','name']
df=spark.createDataFrame(data=data,schema=column)
df.show(truncate=False)

+---+------------------+
|num|name              |
+---+------------------+
|1  |santhi maddipudi  |
|2  |srujan veera      |
|3  |dev veeramachaneni|
+---+------------------+



In [72]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType,IntegerType,LongType

In [74]:
# Define the upper_str function
def upper_str(s):
    split_str = s.split()
    empty_str = ""
    for x in split_str:
        empty_str = empty_str + x[0:1].upper() + x[1:len(x)] + " "
    result = empty_str.strip()
    return result

# Register the upper_str function as a UDF
upper_udf = udf(upper_str, StringType())

# Create a new column based on the existing name column
df = df.withColumn('split column', upper_udf(col('name')))

# Show the DataFrame
df.show()

+---+------------------+------------------+
|num|              name|      split column|
+---+------------------+------------------+
|  1|  santhi maddipudi|  Santhi Maddipudi|
|  2|      srujan veera|      Srujan Veera|
|  3|dev veeramachaneni|Dev Veeramachaneni|
+---+------------------+------------------+

